# BigTable MTLM - dataset perplexity testing

Aim of this notebook:
1. Generate student + question datasets
2. Generate N sets of encounters
3. Report on agreement between these sets

The error between the datasets (for large N) is the inherent probabilistic error in the model
- How does this translate to tolerances in the $\alpha$ and $\delta$ parameters

## Model perplexity
A model $q$ is used to predict the values of a set of samples, $\mathbf{x}$.  Perplexity is defined as:

\\[{perplex}_{q}(\mathbf{x}) = b^{-\frac{1}{N}\Sigma_{i=1}^{N}{log_{b}(q(x_i))}}\\]

Perplexity is a measure of `surprise' as a divergence from the predictions that are seen in the true values.


In [1]:
from collections import defaultdict, Counter
from copy import copy
from math import exp, sqrt, log
from random import random, shuffle, choice, randint, uniform
import numpy
import math

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.constraints import non_neg, max_norm
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda, Flatten
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape, Dropout
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras import constraints

import tensorflow as tf

from utils import generate_student_name
import random

from matplotlib import pyplot as plt




Using TensorFlow backend.


In [2]:
generate_student_name()

'HIYIF SANAB '

In [3]:
sigmoid = lambda z: 1/(1+exp(-z))

class Question():
    def __init__(self, qix, min_diff, max_diff, nt, n_active):
        self.id = qix

        if n_active:
            if len(n_active)==2:
                min_active = n_active[0]
                max_active = n_active[1]
                n_c =  numpy.random.choice(range(min_active, max_active+1))
        else:
            n_c = nt

        choices = numpy.random.choice(range(nt), size=n_c, replace=False)
        not_present= 0#min_diff
        self.betas = [ not_present for _ in range(nt) ]        
  
        for c in choices:
#             self.betas[c] = min_diff
#             self.betas[c] = inv_sigmoid(p) 
#             self.betas[c]=0
            self.betas[c]= uniform(min_diff, max_diff)
    
class Student():
    def __init__(self, ix, min_a, max_a, nt=None):
        self.id = ix
        self.name = generate_student_name()
        n_c = nt
#         n_c = numpy.random.choice(range(int(nt/2),nt+1))
#         n_c = numpy.random.choice(range(1,nt+1))
        choices = numpy.random.choice(range(nt), size=n_c, replace=False)
#         mass = random.uniform(0,(max_a-min_a)*len(choices))

        not_present= 0 #min_a
        
        self.thetas = [ not_present for _ in range(nt) ]        

                
        minp=sigmoid(min_a)
        maxp=sigmoid(max_a)
                
        for c in choices:
#             self.betas[c] = min_diff
#             assume_b = 6#uniform(1,11)
#             assume_pass = uniform(.0001**(1/n_c), .9999**(1/n_c))
# #             p = random.uniform(.0001**(1/n_c), .9999**(1/n_c))
# #             p = random.uniform(minp, maxp)
# #             self.thetas[c] = inv_sigmoid(p) + 6
# #             self.thetas[c] = 0
#             z = inv_sigmoid(assume_pass)
#             th = z + assume_b
#             self.thetas[c]= th
            self.thetas[c] = uniform(min_a, max_a)

In [4]:
def attempt_q(student: Student, q: Question):
    p = calculate_pass_probability(student.thetas, q.betas)
    this_att = uniform(0,1)
    if (this_att <= p):
        passed=1
#         print("passed")
#         student.mastery[q.id] = 1
    else:
        passed=0

    return p,passed

In [5]:
def calculate_pass_probability(thetas, betas):
    p_pass = 1.0
    for th,b in zip(thetas,betas):
#         if b==0:
#             p_pass_step=1.0
#         else:
#             if th==0:
# #                 print("blocking component, ret 0")
#                 return 0
#             else:
        z = (th-b)
        p_pass_step = 1.0 / (1.0 + exp(-z))
# #                 print(th,"vs",b,": ", p_pass_step)
        p_pass *= p_pass_step # simple conjunctive model of success!
    try:
        pass
#         print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [6]:
def create_qs(n_qs, nt, active_limits, beta_min, beta_max):

    max_mag = sqrt((beta_max**2)*nt)
    min_mag = sqrt((beta_min**2)*nt)
    print("Vector length limits:",min_mag,max_mag)

    master_qs = [Question(qix, beta_min,beta_max, nt, active_limits) for qix in range(n_qs)]
    mags = []
    no_comps = []
    for q in master_qs:
        comps = [c for c in q.betas if c>0]
        mag = sqrt(sum([ pow(b, 2) for b in comps ]))
        print("Q:{}, difficulty={:.2f} across {} components".format(q.id, mag, len(comps)))
        mags.append(mag)
        no_comps.append(len(comps))
    
    plt.hist(mags)
    plt.show()
    plt.hist(no_comps)
    plt.show()
    
    for q in master_qs:
        print("qid",q.id,q.betas)
    return master_qs

In [7]:
def create_students(n_students, nt, theta_min, theta_max):

    psi_list = [ Student(psix, theta_min,theta_max, nt=nt) for psix in range(n_students)]
    mags = []
    for psi in psi_list[0:30]:
#         print(psi.name, psi.thetas)
        comps = [c for c in psi.thetas if c>0]
        mag = sqrt(sum([ pow(b, 2) for b in comps ]))
        print("{}, skill={:.2f} across {} comps".format(psi.name, mag, len(comps)))
        mags.append(mag)
    
 ################ PLOTs follow

    fig,ax = plt.subplots(1,2)
    fig.set_size_inches(20,10)
    
    ax[0].hist(mags)
    
    if nt >1:
        itemz = array([ s.thetas for s in psi_list ])
    #     fig.set_size_inches(10, 10)
        ax[1].scatter(itemz[:,0], itemz[:,1], alpha=0.2)
        for i, txt in enumerate(itemz):
            ax[1].annotate(i, (itemz[i,0], itemz[i,1]))
        plt.show()
    
    return psi_list

# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [8]:
import gc
def generate_attempts(master_qs, psi_list):
    attempts =[]
    attempts_by_q = {}
    attempts_by_psi = {}
    attempt_n_map = Counter()

    user_budget = math.inf
    user_patience = 10 #math.inf
    pass_to_remove = True
    
    passes=0
    for run in range(1):
        print("----{}\n".format(run))
        for psi in psi_list:
            spend=0
            qs = [ix for ix in range(len(master_qs))]
            while qs:
                qix = random.choice(qs)
                q = master_qs[qix]
                passed=0

                if psi.name not in attempts_by_psi:
                    attempts_by_psi[psi.name]=[]

                if q not in attempts_by_q:
                    attempts_by_q[q]=[]

                att = 0
#                 while (not passed) and att<user_patience:
#                     pp,passed = attempt_q(psi, q)
#                     tup = (psi.id, q.id, passed, passed)
#                     attempt_n_map[(q.id,psi.id)] += 1
#                     attempts.append(tup)
#                     print("p_pass was",pp,"=",passed) #, "run p:", 1-(1-pp)**max_atts)
#                     attempts_by_psi[psi.name].append(tup)
#                     attempts_by_q[q].append(tup)
#                     att += 1
#                 if (not pass_to_remove) or (pass_to_remove and passed):
#                     qs.remove(qix)
                pp,passed = attempt_q(psi, q)
                if passed:
                    passes+=1
                tup = (psi.id, q.id, passed, passed)
                attempt_n_map[(q.id,psi.id)] += 1
                attempts.append(tup)
#                 print("p_pass was",pp,"=",passed) #, "run p:", 1-(1-pp)**max_atts)
                attempts_by_psi[psi.name].append(tup)
                attempts_by_q[q].append(tup)
                att += 1
                qs.remove(qix)
#                 print("len qs is", len(qs))
    gc.collect()
    print("passed {}/{}".format(passes,len(attempts)))
    return attempts, attempts_by_q, attempts_by_psi, attempt_n_map

In [39]:
%%capture
from IPython.display import clear_output

serieses = []
min_errs = []
n_qs = 100
n_students = 100
n_epochs = 10

# desired confusion prop/n mx
# .5  0 
#  0 .5

def gen_run(n_traits, minth, maxth, minb, maxb, min_active_traits, max_active_traits):
    qs = create_qs(n_qs, n_traits, (min_active_traits, max_active_traits), minb, maxb)
    #     qs, q_table = create_qs_from_blobs(n_qs, 2, n_traits)
    ss = create_students(n_students, n_traits, minth, maxth)

    x = []

    for _ in range(1):
        xa, _,_,_ = generate_attempts(qs,ss) # this is our x list of samples
        x.extend(xa)

    tp,fp,tn,fn=0,0,0,0
    base = 2
    summa=0
    N = len(x)
    for tup in x:
        (psi_id, q_id, passed, passed) = tup
        p = calculate_pass_probability(ss[psi_id].thetas, qs[q_id].betas)
        summa += log((p if passed else (1-p)), base)

        pp = uniform(0,1)
        if pp <= p:
            if passed:
                tp+=1
            else:
                fp+=1
        else:
            if passed:
                fn+=1
            else:
                tn+=1

    acc = (tp+tn)/len(x)
    print("model acc:",acc)
    print(tp,fp)
    print(fn,tn)

    ppx = pow( base, (-summa/N))
    print("perplexity is {}".format(ppx))
    return (fn + fp) + abs(tp-tn), tp,tn,fp,fn
    
dims_scores = {}
param_freedom = 10
random.seed()
seen = set()
mini = 1
maxi = 100
#dimslist = [1,2,3,5,10,25,100]:
for dims in [100]:
    i=0
    while i < 100:
#         minb = 1
#         maxb = uniform(minb,minb+maxi)
#         minth = uniform(minb,maxb)
#         maxth = uniform(minth,minth+maxi)
        
        minb = uniform(mini,maxi)
        maxb = uniform(minb,minb+maxi)
        minth = uniform(minb,maxb+maxi)
        maxth = uniform(maxb+maxi,maxb+2*maxi)
    
        if (dims,minb,maxb,minth,maxth) in seen:
            continue
        clear_output()
        seen.add((dims,minb,maxb,minth,maxth))
        i+=1
        print(">>>",i)
        outz = gen_run(dims, minth, maxth, minb, maxb, dims, dims)
        loss, losstup = outz[0], outz[1:]
        if (dims not in dims_scores) or (dims_scores[dims][0] > loss):
            dims_scores[dims] = (loss, losstup, minth,maxth,minb,maxb)

In [40]:
clear_output()
for dim in dims_scores:
    tup = dims_scores[dim]
    print(dim, tup)

100 (1930, (4972, 4035, 473, 520), 174.41967776840482, 334.3340155161033, 94.16026461348221, 187.59829029717753)
